# Разбиение обучающего датасета на train, oot_val, oof_val

- oot_val - выделим k последних месяцев из обучающего датасета, чтобы его не видела модель при обучении и оценить модель предсказывать в будущем

- oof_val - стратифицировано по таргету и региону выделим небольшой процент, чтобы оценить обобщающую способность модели



In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from pandas.tseries.offsets import MonthEnd
import gc
if 'drive' not in os.listdir():
    from google.colab import drive
    drive.mount('/content/drive')

path_to_folder = '/content/drive/MyDrive/psb_hack'

Mounted at /content/drive


## Загрузка данных и оценка распределения во времени

In [2]:
# чтение данных
train_data = pd.read_excel(os.path.join(path_to_folder,'data','train.xlsx'),index_col=0)
test_data = pd.read_excel(os.path.join(path_to_folder,'data','test.xlsx'),index_col=0)
# подготовка таргета
train_data['target'] = (train_data['Дата отмены'].isna() == False).astype(int)
# получение месяца бронирования
train_data['mon'] = pd.to_datetime(train_data['Дата бронирования'].dt.date, format="%Y%m") + MonthEnd(1)

In [3]:
print('Распределение по месяцам')
display(train_data['mon'].value_counts().sort_index())
print('Распределение по месяцам целевой переменной')
display(train_data.groupby(['mon'])['target'].mean().sort_index())

Распределение по месяцам


,count
mon,
2022-01-31,1030
2022-02-28,1101
2022-03-31,1540
2022-04-30,1785
2022-05-31,1157
2022-06-30,1174
2022-07-31,1077
2022-08-31,1087
2022-09-30,969


Распределение по месяцам целевой переменной


,target
mon,
2022-01-31,0.276699
2022-02-28,0.248865
2022-03-31,0.192857
2022-04-30,0.199440
2022-05-31,0.217805
2022-06-30,0.167802
2022-07-31,0.165274
2022-08-31,0.206992
2022-09-30,0.242518


## Разбиение по времени бронирования

In [4]:
oot_border_mon = '2023-09-30'
oot_val = train_data[train_data['Дата бронирования'] > oot_border_mon]
train = train_data[train_data['Дата бронирования'] <= oot_border_mon]
print('Доля oot_val:',len(oot_val) / (len(oot_val) + len(train)))

Доля oot_val: 0.11339497210972721


## Разбиение стратифицированно по группам

In [5]:
cv = StratifiedGroupKFold(n_splits=8,shuffle=True, random_state=42)
train_index, oof_val_index = list(cv.split(train,train['target'],train['Гостиница']))[0]
oof_val = train.iloc[oof_val_index]
train = train.iloc[train_index]
print('Доля oot_val:',len(oof_val) / (len(oof_val) + len(train)))

Доля oot_val: 0.31108334051538394


## Сохранение

In [6]:
print('Размер full train',len(train_data))
print('Размер train:',len(train))
print('Размер oot_val:',len(oot_val))
print('Размер oof_val:',len(oof_val))

Размер full train 26174
Размер train: 15987
Размер oot_val: 2968
Размер oof_val: 7219


In [7]:
# сохраним все в csv, чтобы быстрее было считывать
train_data.to_csv(os.path.join(path_to_folder,'data','full_train.csv'),header = True,index=False)
train.to_csv(os.path.join(path_to_folder,'data','train.csv'),header = True,index=False)
oot_val.to_csv(os.path.join(path_to_folder,'data','oot_val.csv'),header = True,index=False)
oof_val.to_csv(os.path.join(path_to_folder,'data','oof_val.csv'),header = True,index=False)
test_data.to_csv(os.path.join(path_to_folder,'data','test.csv'),header = True,index=False)